In [1]:
!uv venv rasa_env --python 3.8

Using CPython 3.8.20
Creating virtual environment at: rasa_env
Activate with: rasa_env\Scripts\activate


In [33]:
.\rasa_env\Scripts\activate.bat

SyntaxError: invalid syntax (3343848333.py, line 1)

In [13]:
!uv pip install rasa nest-asyncio ipython --quiet

In [15]:
import rasa
print("Rasa version:", rasa.__version__)
!rasa init --no-prompt

Rasa version: 3.5.17


  File "C:\Users\Gadge\anaconda3\envs\roboticroman\Scripts\rasa.exe", line 1
SyntaxError: Non-UTF-8 code starting with '\xc0' in file C:\Users\Gadge\anaconda3\envs\roboticroman\Scripts\rasa.exe on line 2, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details


In [18]:
!mkdir data

In [19]:
%%writefile data/nlu.yml

version: "3.1"

nlu:
- intent: greet
  examples: |
    - hey
    - hello
    - hi
    - good morning
    - good evening
    - hello there
    - hi there

- intent: goodbye
  examples: |
    - bye
    - goodbye
    - see you later
    - talk to you later
    - catch you later

- intent: ask_programs
  examples: |
    - what programs do you offer?
    - what courses are available?
    - tell me about your degrees
    - what can I study here?
    - list of programs
    - available courses

- intent: ask_admission
  examples: |
    - how to apply?
    - admission process
    - how can I enroll?
    - requirements for admission
    - what documents do I need?
    - application procedure

- intent: ask_location_contact
  examples: |
    - where is the campus?
    - what is the address?
    - contact number
    - phone number
    - email address
    - how to contact the university

- intent: thank_you
  examples: |
    - thanks
    - thank you
    - thanks a lot
    - appreciate it
    

Writing data/nlu.yml


In [20]:
%%writefile data/stories.yml
version: "3.1"

stories:

- story: happy path - greet + ask programs
  steps:
  - intent: greet
  - action: utter_greet
  - intent: ask_programs
  - action: utter_programs

- story: greet + admission
  steps:
  - intent: greet
  - action: utter_greet
  - intent: ask_admission
  - action: utter_admission

- story: greet + location
  steps:
  - intent: greet
  - action: utter_greet
  - intent: ask_location_contact
  - action: utter_location_contact

- story: thank you path
  steps:
  - intent: thank_you
  - action: utter_you_are_welcome
  - intent: goodbye
  - action: utter_goodbye

- story: goodbye directly
  steps:
  - intent: goodbye
  - action: utter_goodbye


Writing data/stories.yml


In [21]:
%%writefile data/rules.yml
version: "3.1"

rules:

- rule: Say goodbye anytime
  steps:
  - intent: goodbye
  - action: utter_goodbye

- rule: Say thank you response
  steps:
  - intent: thank_you
  - action: utter_you_are_welcome


Writing data/rules.yml


In [22]:
%%writefile domain.yml
version: "3.1"

intents:
  - greet
  - goodbye
  - ask_programs
  - ask_admission
  - ask_location_contact
  - thank_you

responses:
  utter_greet:
    - text: "Hello! Welcome to the University Helpdesk 🤖 How can I assist you today?"

  utter_goodbye:
    - text: "Goodbye! Have a great day! 👋"
    - text: "See you later! Feel free to come back anytime."

  utter_programs:
    - text: "We offer the following programs:\n- Bachelor of Business Administration (BBA)\n- Bachelor of Computer Science\n- Master of Business Administration (MBA)\n- Master of Data Science\n- MSc in International Marketing\nWould you like details about any specific program?"

  utter_admission:
    - text: "To apply:\n1. Visit our website and fill the online application\n2. Submit transcripts, passport copy, CV, and motivation letter\n3. Pay the application fee of €50\n4. Attend an interview (online)\nThe next intake is January 2026.\nDo you need the application link?"

  utter_location_contact:
    - text: "Our campus is located in Berlin, Germany 📍\nAddress: Potsdamer Straße 180, 10783 Berlin\nPhone: +49 30 585 80 90\nEmail: info@university.edu"

  utter_you_are_welcome:
    - text: "You're very welcome! 😊"
    - text: "Happy to help!"

  utter_default:
    - text: "I'm sorry, I didn't quite understand that. Could you rephrase? I'm here to help with programs, admissions, or contact info."

actions:
  - utter_greet
  - utter_goodbye
  - utter_programs
  - utter_admission
  - utter_location_contact
  - utter_you_are_welcome
  - utter_default


Writing domain.yml


In [23]:
%%writefile config.yml
language: en

pipeline:
  - name: WhitespaceTokenizer
  - name: RegexFeaturizer
  - name: LexicalSyntacticFeaturizer
  - name: CountVectorsFeaturizer
  - name: CountVectorsFeaturizer
    analyzer: char_wb
    min_ngram: 1
    max_ngram: 4
  - name: DIETClassifier
    epochs: 100
  - name: EntitySynonymMapper
  - name: ResponseSelector
    epochs: 100

policies:
  - name: MemoizationPolicy
  - name: RulePolicy
  - name: TEDPolicy
    max_history: 5
    epochs: 100
  - name: FallbackPolicy
    nlu_threshold: 0.4
    core_threshold: 0.3
    fallback_action_name: utter_default

Writing config.yml


In [26]:
!rasa train

  File "C:\Users\Gadge\anaconda3\envs\roboticroman\Scripts\rasa.exe", line 1
SyntaxError: Non-UTF-8 code starting with '\xc0' in file C:\Users\Gadge\anaconda3\envs\roboticroman\Scripts\rasa.exe on line 2, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details


In [21]:
import json
import yaml

with open('data/faqs.json', 'r') as f:
    faqs = json.load(f)

nlu_data = {'version': '3.1', 'nlu': []}
# general_faq = {'intent': 'faq', 'examples': ['What is BSBI?', 'Tell me about fees', 'Visa help?']}
# nlu_data['nlu'].append(general_faq)

for faq in faqs:
    print(f"- intent: faq")
    print("  examples: |")
    print(f"   - {faq['question']}")
    print(f"   - {faq['question'].lower()}")
    print("  responses: |")
    print(f"   - {faq['answer']}\n")


- intent: faq
  examples: |
   - Is BSBI a university?
   - is bsbi a university?
  responses: |
   - No, the Berlin School of Business and Innovation (BSBI) is not a university. It is a private higher education institution that offers undergraduate, postgraduate and doctorate programmes in partnership with accredited universities. BSBI delivers the teaching and student support, while the degrees are officially awarded by its partner universities.

- intent: faq
  examples: |
   - Are BSBI programmes recognised and accredited?
   - are bsbi programmes recognised and accredited?
  responses: |
   - Yes. All degrees awarded at BSBI come from accredited and recognized partner universities. These universities are officially recognized in their respective home countries and are internationally renowned. The current partner universities are: Università Telematica Internazionale UNINETTUNO (Italy) – state-recognised and accredited by the Italian Ministry of Education. Concordia University Chi